In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 500
BATCH_SIZE=100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print('다음 장치를 사용합니다:', DEVICE)

다음 장치를 사용합니다: cpu


In [3]:
trainset = datasets.FashionMNIST('./.data',
                                train=True,
                                download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,),(0.5,))
                                ]))
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [4]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embed = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(110, 256),    # 110 = 100(무작위 텐서의 크기) + 10(무작위 레이블 수)
            nn.LeakyReLU(0.2, inplace=True),   # inplace=True : 입력을 복사하지 않고 바로 조작
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
        
    def forward(self, z, labels):
        c = self.embed(labels)  # 클래스 레이블
        x = torch.cat([z,c], 1)  # 무작위 벡터와 클래스 레이블을 이어붙임
        return self.model(x)

In [9]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embed = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(794, 1024),    # 794 = 784(이미지 크기) + 10(레이블 수)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()     # 0~1 값 반환
        )
        
    def forward(self, x, labels):
        c = self.embed(labels)
        x = torch.cat([x, c], 1)
        return self.model(x)

In [10]:
D = Discriminator().to(DEVICE)
G = Generator().to(DEVICE)

In [11]:
# 오차함수 : 이진 교차 엔트로피
criterion = nn.BCELoss()
 
# 최적화 알고리즘 : Adam
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
total_step = len(train_loader)

for epoch in range(EPOCHS):  # GAN 학습
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)
        
        # discriminator가 진짜 이미지를 진짜로 인식하는 오차 계산 (레이블 입력)
        labels = labels.to(DEVICE)
        outputs = D(images, labels)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # 무작위 텐서와 무작위 레이블을 생성자에 입력해 가짜 이미지 생성
        z = torch.randn(BATCH_SIZE, 100).to(DEVICE)
        g_label = torch.randint(0,10, (BATCH_SIZE,)).to(DEVICE)
        fake_images = G(z, g_label)
        
        outputs = D(fake_images, g_label)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Discriminator의 오차 구함
        d_loss = d_loss_real + d_loss_fake
        
        # Discriminator 학습
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Generator가 Discriminator를 속였는지에 대한 오차 계산 (무작위 레이블 입력)
        fake_images = G(z, g_label)
        outputs = D(fake_images, g_label)
        g_loss = criterion(outputs, real_labels)
        
        # Generator 학습
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    # 학습 진행 알아보기
    # D(x): 진짜를 진짜로 인식, D(G(z)): 가짜를 진짜로 인식
    print('epoch [{}/{}] d_loss:{:.4f} g_loss:{:.4f} D(x):{:.2f} D(G(z)):{:.2f}'.format(epoch, EPOCHS, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))

epoch [0/500] d_loss:0.4086 g_loss:4.2491 D(x):0.93 D(G(z)):0.18
epoch [1/500] d_loss:0.5245 g_loss:3.3247 D(x):0.88 D(G(z)):0.20
epoch [2/500] d_loss:0.3712 g_loss:4.4737 D(x):0.96 D(G(z)):0.18
epoch [3/500] d_loss:0.4974 g_loss:6.4356 D(x):0.83 D(G(z)):0.06
epoch [4/500] d_loss:0.3505 g_loss:3.3956 D(x):0.88 D(G(z)):0.10
epoch [5/500] d_loss:0.7940 g_loss:2.4891 D(x):0.83 D(G(z)):0.28
epoch [6/500] d_loss:0.4003 g_loss:2.7625 D(x):0.84 D(G(z)):0.12
epoch [7/500] d_loss:0.7553 g_loss:2.0209 D(x):0.82 D(G(z)):0.28
epoch [8/500] d_loss:0.7768 g_loss:2.0918 D(x):0.73 D(G(z)):0.21
epoch [9/500] d_loss:0.7210 g_loss:1.7082 D(x):0.77 D(G(z)):0.26
epoch [10/500] d_loss:0.8232 g_loss:1.7130 D(x):0.70 D(G(z)):0.23
epoch [11/500] d_loss:1.0734 g_loss:1.5980 D(x):0.64 D(G(z)):0.29
epoch [12/500] d_loss:0.7543 g_loss:1.7847 D(x):0.76 D(G(z)):0.24
epoch [13/500] d_loss:0.9348 g_loss:1.6716 D(x):0.70 D(G(z)):0.32
epoch [14/500] d_loss:0.7871 g_loss:2.3026 D(x):0.75 D(G(z)):0.23
epoch [15/500] d_los

In [ ]:
# label : 이름
# 0: 티셔츠
# 1: 바지
# 2: 스웨터
# 3: 드레스
# 4: 코트
# 5:샌들
# 6: 셔츠
# 7: 신발
# 8: 가방
# 9: 부츠

In [ ]:
# 만들고 싶은 아이템 생성하여 시각화
item_number = 9  # label 번호
z = torch.randn(1, 100).to(DEVICE)   # 배치 크기 1
g_label = torch.full((1,), item_number, dtype=torch.long).to(DEVICE)   # torch.full : 새로운 텐서를 만드는 함수, 1번째 인자: 텐서 크기, 2번째 인자: 텐서를 초기화할 값

sample_images = G(z, g_label)

sample_images_img = np.reshape(sample_images.data.cpu().numpy()[0], (28,28))   # 텐서를 시각화하기 위해 numpy 행렬로 바꿈
plt.imshow(sample_images_img, cmap='gray')
plt.show()